In [1]:
import pandas as pd
import json

In [2]:
df_world_finbert= pd.read_csv('../Data/2_Processed/Unstructured_Data/Global/world_finbert.csv')
df_politics_finbert= pd.read_csv('../Data/2_Processed/Unstructured_Data/Global/politics_finbert.csv')
df_coronavirus_finbert= pd.read_csv('../Data/2_Processed/Unstructured_Data/Global/coronavirus_finbert.csv')
df_aapl_finbert= pd.read_csv('../Data/2_Processed/Unstructured_Data/Stock/aapl_finbert.csv')
df_meta_finbert= pd.read_csv('../Data/2_Processed/Unstructured_Data/Stock/meta_finbert.csv')
df_tsla_finbert= pd.read_csv('../Data/2_Processed/Unstructured_Data/Stock/tsla_finbert.csv')

df_world_flair= pd.read_csv('../Data/2_Processed/Unstructured_Data/Global/world_flair.csv')
df_politics_flair= pd.read_csv('../Data/2_Processed/Unstructured_Data/Global/politics_flair.csv')
df_coronavirus_flair= pd.read_csv('../Data/2_Processed/Unstructured_Data/Global//coronavirus_flair.csv')
df_aapl_flair= pd.read_csv('../Data/2_Processed/Unstructured_Data/Stock/aapl_flair.csv')
df_meta_flair= pd.read_csv('../Data/2_Processed/Unstructured_Data/Stock/meta_flair.csv')
df_tsla_flair= pd.read_csv('../Data/2_Processed/Unstructured_Data/Stock/tsla_flair.csv')

df_world_vader= pd.read_csv('../Data/2_Processed/Unstructured_Data/Global/world_vader.csv')
df_politics_vader= pd.read_csv('../Data/2_Processed/Unstructured_Data/Global/politics_vader.csv')
df_coronavirus_vader= pd.read_csv('../Data/2_Processed/Unstructured_Data/Global/coronavirus_vader.csv')
df_aapl_vader= pd.read_csv('../Data/2_Processed/Unstructured_Data/Stock/aapl_vader.csv')
df_meta_vader= pd.read_csv('../Data/2_Processed/Unstructured_Data/Stock/meta_vader.csv')
df_tsla_vader= pd.read_csv('../Data/2_Processed/Unstructured_Data/Stock/tsla_vader.csv')

In [3]:
import yfinance as yf

def downloadHistory(tickerName, start="2018-01-01", end="2022-09-30"):
    return yf.download(tickerName, start=start, end=end)[['Open', 'High', 'Low', 'Close', 'Volume']]

In [4]:
aapl_hist = downloadHistory('AAPL')
meta_hist = downloadHistory('META')
tsla_hist = downloadHistory('TSLA')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
aapl_keyword = ['aapl', 'apple', 'iphone', 'macbook', 'ipad', 'ios']
meta_keyword = ['fb', 'facebook', 'meta', 'metaverse']
tsla_keyword = ['tsla', 'tesla', 'ev', 'elon', 'musk', 'electric car']

df_aapl_finbert_filtered = df_aapl_finbert[df_aapl_finbert['Processed Title'].str.contains('|'.join(aapl_keyword)) == True]
df_meta_finbert_filtered = df_meta_finbert[df_meta_finbert['Processed Title'].str.contains('|'.join(meta_keyword)) == True]
df_tsla_finbert_filtered = df_tsla_finbert[df_tsla_finbert['Processed Title'].str.contains('|'.join(tsla_keyword)) == True]

df_aapl_flair_filtered = df_aapl_flair[df_aapl_flair['Processed Title'].str.contains('|'.join(aapl_keyword)) == True]
df_meta_flair_filtered = df_meta_flair[df_meta_flair['Processed Title'].str.contains('|'.join(meta_keyword)) == True]
df_tsla_flair_filtered = df_tsla_flair[df_tsla_flair['Processed Title'].str.contains('|'.join(tsla_keyword)) == True]

df_aapl_vader_filtered = df_aapl_vader[df_aapl_vader['Processed Title'].str.contains('|'.join(aapl_keyword)) == True]
df_meta_vader_filtered = df_meta_vader[df_meta_vader['Processed Title'].str.contains('|'.join(meta_keyword)) == True]
df_tsla_vader_filtered = df_tsla_vader[df_tsla_vader['Processed Title'].str.contains('|'.join(tsla_keyword)) == True]

In [6]:
def sentiment_aggregator(df, title = True, type = "abs_max"):
    """
    Aggregates sentiments on a per day basis.

    Parameters
    ----------
    df: DataFrame
        Dataset generated after sentiment analysis.
    title: boolean
        To indicate if the news title or news body text is used to generate the aggregated sentiment. 
        Default is True (ie. News title is used for aggregated sentiment)
    type: Str {"mean", "abs_max"}
        To indicate method of calculation.
        "mean": Group by Date and takes mean of "Compound"
        "abs_max": Calculates the absolute max of "Positive" and "Negative" column. Then group by Date and takes mean of this new column

    Returns
    -------
    Output : Series
        Contains aggregated sentiment for each day
    """
    if title:
        positive = "Positive_Title"
        negative = "Negative_Title"
    else:
        positive = "Positive_Text"
        negative = "Negative_Text"

    if type == "mean":
        df["Compound"] = df[positive] - df[negative]
        return df.groupby('Date')['Compound'].aggregate('mean')

    elif type == "abs_max":
        df['Negative'] = -df[negative]
        df['Positive'] = df[positive]
        df['Sentiment'] = df.apply(lambda x: max(x["Negative"], x["Positive"], key=abs), axis=1)

        return df.groupby('Date')['Sentiment'].aggregate('mean')


# Finbert
meta_finbert_maxabs_df = sentiment_aggregator(df_meta_finbert_filtered, title = True, type = "abs_max") 
aapl_finbert_maxabs_df = sentiment_aggregator(df_aapl_finbert_filtered, title = True, type = "abs_max") 
tsla_finbert_maxabs_df = sentiment_aggregator(df_tsla_finbert_filtered, title = True, type = "abs_max") 

meta_finbert_mean_df = sentiment_aggregator(df_meta_finbert_filtered, title = True, type = "mean") 
aapl_finbert_mean_df = sentiment_aggregator(df_aapl_finbert_filtered, title = True, type = "mean") 
tsla_finbert_mean_df = sentiment_aggregator(df_tsla_finbert_filtered, title = True, type = "mean") 

df_world_finbert_maxabs = sentiment_aggregator(df_world_finbert, title = True, type = "abs_max") 
df_politics_finbert_maxabs = sentiment_aggregator(df_politics_finbert, title = True, type = "abs_max") 
df_coronavirus_finbert_maxabs = sentiment_aggregator(df_coronavirus_finbert, title = True, type = "abs_max") 

df_world_finbert_mean = sentiment_aggregator(df_world_finbert, title = True, type = "mean") 
df_politics_finbert_mean = sentiment_aggregator(df_politics_finbert, title = True, type = "mean") 
df_coronavirus_finbert_mean = sentiment_aggregator(df_coronavirus_finbert, title = True, type = "mean") 


C:\Users\65831\AppData\Local\Temp\ipykernel_22268\3221107698.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Negative'] = -df[negative]
C:\Users\65831\AppData\Local\Temp\ipykernel_22268\3221107698.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Positive'] = df[positive]
C:\Users\65831\AppData\Local\Temp\ipykernel_22268\3221107698.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [7]:
def sentiment_aggregator(df, title = True):
    # flair only gives one value and a label (POSITIVE or NEGATIVE) so we just use mean
    # if title:
    return df.groupby('Date')['Score'].aggregate('mean')

    # else:
    #     return df.groupby('Date')['Score Text'].aggregate('mean')
        
        
# Flair
meta_flair_df = sentiment_aggregator(df_meta_flair_filtered, title = True) 
aapl_flair_df = sentiment_aggregator(df_aapl_flair_filtered, title = True) 
tsla_flair_df = sentiment_aggregator(df_tsla_flair_filtered, title = True) 

df_world_flair = sentiment_aggregator(df_world_flair, title = True) 
df_politics_flair = sentiment_aggregator(df_politics_flair, title = True) 
df_coronavirus_flair = sentiment_aggregator(df_coronavirus_flair, title = True) 



In [8]:
# Vader
def sentiment_aggregator(df, title = True, type="mean"):
    """
    Aggregates sentiments on a per day basis.

    Parameters
    ----------
    df: DataFrame
        Dataset generated after sentiment analysis.
    title: boolean
        To indicate if the news title or news body text is used to generate the aggregated sentiment. 
        Default is True (ie. News title is used for aggregated sentiment)
    type: Str {"mean", "abs_max"}
        To indicate method of calculation.
        "mean": Group by Date and takes mean of "Compound"
        "abs_max": Calculates the absolute max of "Positive" and "Negative" column. Then group by Date and takes mean of this new column

    Returns
    -------
    Output : Series
        Contains aggregated sentiment for each day
    """
   
    target = "Sentiment Title"
    #  if title else "Sentiment Text"
    df = df.copy()
    df[target] = df[target].str.replace('\'','\"')
    df[target] = df[target].apply(lambda x: json.loads(x))

    df['Negative'] = df[target].apply(lambda x: x.get('neg'))
    df['Neutral'] = df[target].apply(lambda x: x.get('neu'))
    df['Positive'] = df[target].apply(lambda x: x.get('pos'))
    df['Compound'] = df[target].apply(lambda x: x.get('compound'))

    if type == "mean":
        return df.groupby('Date')['Compound'].aggregate('mean')

    elif type == "abs_max":
        df['Negative'] = df[target].apply(lambda x: -x.get('neg'))
        df['Sentiment'] = df.apply(lambda x: max(x['Negative'], x['Positive'], key=abs), axis=1)

        return df.groupby('Date')['Sentiment'].aggregate('mean')


# Vader
meta_vader_maxabs_df = sentiment_aggregator(df_meta_vader_filtered, title = True, type = "abs_max") 
aapl_vader_maxabs_df = sentiment_aggregator(df_aapl_vader_filtered, title = True, type = "abs_max") 
tsla_vader_maxabs_df = sentiment_aggregator(df_tsla_vader_filtered, title = True, type = "abs_max") 

meta_vader_mean_df = sentiment_aggregator(df_meta_vader_filtered, title = True, type = "mean") 
aapl_vader_mean_df = sentiment_aggregator(df_aapl_vader_filtered, title = True, type = "mean") 
tsla_vader_mean_df = sentiment_aggregator(df_tsla_vader_filtered, title = True, type = "mean") 

df_world_vader_absmax = sentiment_aggregator(df_world_vader, title = True, type = "abs_max") 
df_politics_vader_absmax = sentiment_aggregator(df_politics_vader, title = True, type = "abs_max") 
df_coronavirus_vader_absmax = sentiment_aggregator(df_coronavirus_vader, title = True, type = "abs_max") 

df_world_vader_mean = sentiment_aggregator(df_world_vader, title = True, type = "mean") 
df_politics_vader_mean = sentiment_aggregator(df_politics_vader, title = True, type = "mean") 
df_coronavirus_vader_mean = sentiment_aggregator(df_coronavirus_vader, title = True, type = "mean") 


In [9]:
meta_finbert_maxabs_df.index = pd.to_datetime(meta_finbert_maxabs_df.index)
aapl_finbert_maxabs_df.index = pd.to_datetime(aapl_finbert_maxabs_df.index)
tsla_finbert_maxabs_df.index = pd.to_datetime(tsla_finbert_maxabs_df.index)

meta_flair_df.index = pd.to_datetime(meta_flair_df.index)
aapl_flair_df.index = pd.to_datetime(aapl_flair_df.index)
tsla_flair_df.index = pd.to_datetime(tsla_flair_df.index)

meta_vader_maxabs_df.index = pd.to_datetime(meta_vader_maxabs_df.index)
aapl_vader_maxabs_df.index = pd.to_datetime(aapl_vader_maxabs_df.index)
tsla_vader_maxabs_df.index = pd.to_datetime(tsla_vader_maxabs_df.index)

meta_finbert_mean_df.index = pd.to_datetime(meta_finbert_mean_df.index)
aapl_finbert_mean_df.index = pd.to_datetime(aapl_finbert_mean_df.index)
tsla_finbert_mean_df.index = pd.to_datetime(tsla_finbert_mean_df.index)

meta_vader_mean_df.index = pd.to_datetime(meta_vader_mean_df.index)
aapl_vader_mean_df.index = pd.to_datetime(aapl_vader_mean_df.index)
tsla_vader_mean_df.index = pd.to_datetime(tsla_vader_mean_df.index)

df_world_finbert_maxabs.index=pd.to_datetime(df_world_finbert_maxabs.index)
df_politics_finbert_maxabs.index=pd.to_datetime(df_politics_finbert_maxabs.index)
df_coronavirus_finbert_maxabs.index=pd.to_datetime(df_coronavirus_finbert_maxabs.index)

df_world_finbert_mean.index=pd.to_datetime(df_world_finbert_mean.index)
df_politics_finbert_mean.index=pd.to_datetime(df_politics_finbert_mean.index)
df_coronavirus_finbert_mean.index=pd.to_datetime(df_coronavirus_finbert_mean.index)

df_world_flair.index=pd.to_datetime(df_world_flair.index)
df_politics_flair.index=pd.to_datetime(df_politics_flair.index)
df_coronavirus_flair.index=pd.to_datetime(df_coronavirus_flair.index)

df_world_vader_absmax.index=pd.to_datetime(df_world_vader_absmax.index)
df_politics_vader_absmax.index=pd.to_datetime(df_politics_vader_absmax.index)
df_coronavirus_vader_absmax.index=pd.to_datetime(df_coronavirus_vader_absmax.index)

df_world_vader_mean.index=pd.to_datetime(df_world_vader_mean.index)
df_politics_vader_mean.index=pd.to_datetime(df_politics_vader_mean.index)
df_coronavirus_vader_mean.index=pd.to_datetime(df_coronavirus_vader_mean.index)

In [10]:
meta_finbert_maxabs_merge=pd.merge(meta_hist,meta_finbert_maxabs_df, how='left', left_index=True, right_index=True)
aapl_finbert_maxabs_merge=pd.merge(aapl_hist,aapl_finbert_maxabs_df, how='left', left_index=True, right_index=True)
tsla_finbert_maxabs_merge=pd.merge(tsla_hist,tsla_finbert_maxabs_df, how='left', left_index=True, right_index=True)

meta_vader_maxabs_merge=pd.merge(meta_hist,meta_vader_maxabs_df, how='left', left_index=True, right_index=True)
aapl_vader_maxabs_merge=pd.merge(aapl_hist,aapl_vader_maxabs_df, how='left', left_index=True, right_index=True)
tsla_vader_maxabs_merge=pd.merge(tsla_hist,tsla_vader_maxabs_df, how='left', left_index=True, right_index=True)

meta_finbert_mean_merge=pd.merge(meta_hist,meta_finbert_mean_df, how='left', left_index=True, right_index=True)
aapl_finbert_mean_merge=pd.merge(aapl_hist,aapl_finbert_mean_df, how='left', left_index=True, right_index=True)
tsla_finbert_mean_merge=pd.merge(tsla_hist,tsla_finbert_mean_df, how='left', left_index=True, right_index=True)

meta_flair_merge=pd.merge(meta_hist,meta_flair_df, how='left', left_index=True, right_index=True)
aapl_flair_merge=pd.merge(aapl_hist,aapl_flair_df, how='left', left_index=True, right_index=True)
tsla_flair_merge=pd.merge(tsla_hist,tsla_flair_df, how='left', left_index=True, right_index=True)

meta_vader_mean_merge=pd.merge(meta_hist,meta_vader_mean_df, how='left', left_index=True, right_index=True)
aapl_vader_mean_merge=pd.merge(aapl_hist,aapl_vader_mean_df, how='left', left_index=True, right_index=True)
tsla_vader_mean_merge=pd.merge(tsla_hist,tsla_vader_mean_df, how='left', left_index=True, right_index=True)

# world

df_world_finbert_maxabs_merge=pd.merge(meta_hist,df_world_finbert_maxabs, how='left', left_index=True, right_index=True)
df_politics_finbert_maxabs_merge=pd.merge(meta_hist,df_politics_finbert_maxabs, how='left', left_index=True, right_index=True)
df_coronavirus_finbert_maxabs_merge=pd.merge(meta_hist,df_coronavirus_finbert_maxabs, how='left', left_index=True, right_index=True)

df_world_finbert_mean_merge=pd.merge(meta_hist,df_world_finbert_mean, how='left', left_index=True, right_index=True)
df_politics_finbert_mean_merge=pd.merge(meta_hist,df_politics_finbert_mean, how='left', left_index=True, right_index=True)
df_coronavirus_finbert_mean_merge=pd.merge(meta_hist,df_coronavirus_finbert_mean, how='left', left_index=True, right_index=True)

df_world_flair_merge=pd.merge(meta_hist,df_world_flair, how='left', left_index=True, right_index=True)
df_politics_flair_merge=pd.merge(meta_hist,df_politics_flair, how='left', left_index=True, right_index=True)
df_coronavirus_flair_merge=pd.merge(meta_hist,df_coronavirus_flair, how='left', left_index=True, right_index=True)

df_world_vader_maxabs_merge=pd.merge(meta_hist,df_world_vader_absmax, how='left', left_index=True, right_index=True)
df_politics_vader_maxabs_merge=pd.merge(meta_hist,df_politics_vader_absmax, how='left', left_index=True, right_index=True)
df_coronavirus_vader_maxabs_merge=pd.merge(meta_hist,df_coronavirus_vader_absmax, how='left', left_index=True, right_index=True)

df_world_vader_mean_merge=pd.merge(meta_hist,df_world_vader_mean, how='left', left_index=True, right_index=True)
df_politics_vader_mean_merge=pd.merge(meta_hist,df_politics_vader_mean, how='left', left_index=True, right_index=True)
df_coronavirus_vader_mean_merge=pd.merge(meta_hist,df_coronavirus_vader_mean, how='left', left_index=True, right_index=True)




In [11]:
from sklearn.impute import KNNImputer

# f
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(meta_finbert_maxabs_merge)
transformed = imputer.transform(meta_finbert_maxabs_merge)
meta_finbert_maxabs_merge[:]= imputer.transform(meta_finbert_maxabs_merge)

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(aapl_finbert_maxabs_merge)
transformed = imputer.transform(aapl_finbert_maxabs_merge)
aapl_finbert_maxabs_merge[:]= imputer.transform(aapl_finbert_maxabs_merge)

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(tsla_finbert_maxabs_merge)
transformed = imputer.transform(tsla_finbert_maxabs_merge)
tsla_finbert_maxabs_merge[:]= imputer.transform(tsla_finbert_maxabs_merge)

###
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(meta_flair_merge)
transformed = imputer.transform(meta_flair_merge)
meta_flair_merge[:]= imputer.transform(meta_flair_merge)

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(aapl_flair_merge)
transformed = imputer.transform(aapl_flair_merge)
aapl_flair_merge[:]= imputer.transform(aapl_flair_merge)

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(tsla_flair_merge)
transformed = imputer.transform(tsla_flair_merge)
tsla_flair_merge[:]= imputer.transform(tsla_flair_merge)

###
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(meta_vader_maxabs_merge)
transformed = imputer.transform(meta_vader_maxabs_merge)
meta_vader_maxabs_merge[:]= imputer.transform(meta_vader_maxabs_merge)

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(aapl_vader_maxabs_merge)
transformed = imputer.transform(aapl_vader_maxabs_merge)
aapl_vader_maxabs_merge[:]= imputer.transform(aapl_vader_maxabs_merge)

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(tsla_vader_maxabs_merge)
transformed = imputer.transform(tsla_vader_maxabs_merge)
tsla_vader_maxabs_merge[:]= imputer.transform(tsla_vader_maxabs_merge)

###
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(meta_finbert_mean_merge)
transformed = imputer.transform(meta_finbert_mean_merge)
meta_finbert_mean_merge[:]= imputer.transform(meta_finbert_mean_merge)

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(aapl_finbert_mean_merge)
transformed = imputer.transform(aapl_finbert_mean_merge)
aapl_finbert_mean_merge[:]= imputer.transform(aapl_finbert_mean_merge)

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(tsla_finbert_mean_merge)
transformed = imputer.transform(tsla_finbert_mean_merge)
tsla_finbert_mean_merge[:]= imputer.transform(tsla_finbert_mean_merge)

###
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(meta_vader_mean_merge)
transformed = imputer.transform(meta_vader_mean_merge)
meta_vader_mean_merge[:]= imputer.transform(meta_vader_mean_merge)

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(aapl_vader_mean_merge)
transformed = imputer.transform(aapl_vader_mean_merge)
aapl_vader_mean_merge[:]= imputer.transform(aapl_vader_mean_merge)

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(tsla_vader_mean_merge)
transformed = imputer.transform(tsla_vader_mean_merge)
tsla_vader_mean_merge[:]= imputer.transform(tsla_vader_mean_merge)

In [14]:
keys = ["AAPL", "META", "TSLA", "World", "Politics", "Coronavirus"]

lst = [meta_finbert_maxabs_merge['Sentiment'], aapl_finbert_maxabs_merge['Sentiment'], tsla_finbert_maxabs_merge['Sentiment'], df_world_finbert_maxabs_merge['Sentiment'], df_politics_finbert_maxabs_merge['Sentiment'], df_coronavirus_finbert_maxabs_merge['Sentiment']]
df_all_maxabs_finbert = pd.concat(lst, keys=keys, axis=1)
df_all_maxabs_finbert = df_all_maxabs_finbert.sort_values(by="Date")

lst = [meta_finbert_mean_merge['Compound'], aapl_finbert_mean_merge['Compound'], tsla_finbert_mean_merge['Compound'], df_world_finbert_mean_merge['Compound'], df_politics_finbert_mean_merge['Compound'], df_coronavirus_finbert_mean_merge['Compound']]
df_all_mean_finbert = pd.concat(lst, keys=keys, axis=1)
df_all_mean_finbert = df_all_mean_finbert.sort_values(by="Date")

lst = [meta_flair_merge['Score'], aapl_flair_merge['Score'], tsla_flair_merge['Score'], df_world_flair_merge['Score'], df_politics_flair_merge['Score'], df_coronavirus_flair_merge['Score']]
df_all_flair = pd.concat(lst, keys=keys, axis=1)
df_all_flair = df_all_flair.sort_values(by="Date")

lst = [meta_vader_maxabs_merge['Sentiment'], aapl_vader_maxabs_merge['Sentiment'], tsla_vader_maxabs_merge['Sentiment'], df_world_vader_maxabs_merge['Sentiment'], df_politics_vader_maxabs_merge['Sentiment'], df_coronavirus_vader_maxabs_merge['Sentiment']]
df_all_maxabs_vader = pd.concat(lst, keys=keys, axis=1)
df_all_maxabs_vader = df_all_maxabs_vader.sort_values(by="Date")

lst = [meta_vader_mean_merge['Compound'], aapl_vader_mean_merge['Compound'], tsla_vader_mean_merge['Compound'], df_world_vader_mean_merge['Compound'], df_politics_vader_mean_merge['Compound'], df_coronavirus_vader_mean_merge['Compound']]
df_all_mean_vader = pd.concat(lst, keys=keys, axis=1)
df_all_mean_vader = df_all_mean_vader.sort_values(by="Date")

df_all_maxabs_finbert.to_csv('../Data/2_Processed/Unstructured_Data/All/Filtered/all_finbert_maxabs.csv')
df_all_mean_finbert.to_csv('../Data/2_Processed/Unstructured_Data/All/Filtered/all_finbert_mean.csv')
df_all_flair.to_csv('../Data/2_Processed/Unstructured_Data/All/Filtered/all_flair.csv')
df_all_maxabs_vader.to_csv('../Data/2_Processed/Unstructured_Data/All/Filtered/all_vader_maxabs.csv')
df_all_mean_vader.to_csv('../Data/2_Processed/Unstructured_Data/All/Filtered/all_vader_mean.csv')